#### CLPsych19 Shared Task
For questions contact Michelle.Morales@ibm.com

#### Load the data

In [ ]:
import os
import gensim
import pandas as pd
from nltk.tokenize import sent_tokenize, word_tokenize
from gensim.test.utils import get_tmpfile
from gensim.models import Word2Vec, FastText

In [ ]:
# Optional steps depending on where your scripts and data are
# os.getcwd()
# os.chdir('clpsych19_training_data/') # point directory to training data folder

In [ ]:
text_data = pd.read_csv('shared_task_posts.csv')

In [ ]:
text_data.head()

In [ ]:
text_data = text_data.fillna('')
# text_data['title_stokenize'] = text_data.apply(lambda row: sent_tokenize(row['post_title'].lower()), axis=1)

In [ ]:
text_data['title_tokens'] = text_data.apply(lambda row: [word_tokenize(s) for s in sent_tokenize(row['post_title'].lower())], axis=1)

In [ ]:
text_data['post_tokens'] = text_data.apply(lambda row: [word_tokenize(s) for s in sent_tokenize(row['post_body'].lower())], axis=1)

In [ ]:
all_tokens = text_data['title_tokens'].values + text_data['post_tokens'].values

In [ ]:
one_list = []
for a in all_tokens:
    sents = a
    for s in sents:
        one_list.append(s)

In [ ]:
# TODO: more preprocessing

In [ ]:
### TO SLOW
# # Get tokenized sentences from titles
# text_data = text_data.fillna('')
# tokenized_sentences = []
# for (i,row) in text_data.iterrows():
#     title = row['post_title']
#     post = row['post_body']
#     for s in sent_tokenize(title.lower()):
#         tokens = word_tokenize(s)
#         tokenized_sentences.append(tokens)
#     for s in sent_tokenize(post.lower()):
#         tokens = word_tokenize(s)
#         tokenized_sentences.append(tokens)

In [ ]:
one_list[0]

#### Train word vectors

##### Word2vec

In [ ]:
# Train wordvecs using gensim
model = gensim.models.Word2Vec(one_list, size=100, window=5, min_count=5, workers=4, sg=1)

In [ ]:
words = list(model.wv.vocab)
print(len(words))
path = get_tmpfile("word2vec_reddit.model")
model.save("word2vec_reddit.model")
model.wv.save_word2vec_format("word2vec_reddit.txt", binary=False)

##### Fast text

In [ ]:
fmodel = FastText(size=4, window=3, min_count=5, sentences=one_list, iter=10)

In [ ]:
path = get_tmpfile("fasttext_reddit.model")
fmodel.save('fasttext_reddit.model')

In [ ]:
fmodel['suici']

#### Load and visualize trained model

In [ ]:
model = Word2Vec.load("word2vec_reddit.model")

In [ ]:
model.most_similar('depression')

In [ ]:
# Transform data output to make it work with the tensorflow embedding projector: https://projector.tensorflow.org/
words = list(model.wv.vocab)
outF = open("words.tsv", "w")
for w in words:
  # write line to output file
    outF.write(w)
    outF.write("\n")
outF.close()

X = reddit_model[reddit_model.wv.vocab]
outF = open("vectors.tsv", "w")
for vector in X:
  # write line to output file
    outF.write('\t'.join([str(x) for x in vector.tolist()]))
    outF.write("\n")
outF.close()

#### Load retrofitted vectors

In [ ]:
from gensim.models import KeyedVectors
# Retrofit them using code from: https://github.com/mfaruqui/retrofitting
retro_model = KeyedVectors.load_word2vec_format('retro_word2vec_reddit.txt')

In [ ]:
retro_model.most_similar('depression')